In [ ]:
def make_coordinates (image, line_parameters):
  if line_parameters is None or len(line_parameters) < 2:
    return None
  slope, intercept = line_parameters
  y1 = image.shape[0]
  y2 = int(y1*(3/5))
  if slope == 0:
      return None 
  x1 = int((y1-intercept)/slope)
  x2= int((y2-intercept)/slope)
  return np.array([x1, y1, x2, y2])
  
def average_slope_intercept(image, lines):
  left_line = []
  right_line = []
  if lines is None: 
      return None, None
  for line in lines:
    x1, y1, x2, y2= line.reshape(4)
    if x2 - x1 == 0:
        continue
    parameters = np.polyfit((x1, x2), (y1, y2), 1)
    slope = parameters[0]
    intercept = parameters[1]
    if slope < -0.3:
      left_line.append((slope, intercept))
    elif slope > 0.3:
      right_line.append((slope, intercept))
  if left_line:
      left_line_average = np.average(left_line, axis=0)
  else:
      None
  if right_line:
      right_line_average = np.average(right_line, axis = 0)
  else:
      None
      
  final_left_line = make_coordinates(image, left_line_average)
  final_right_line = make_coordinates(image, right_line_average)

  return final_left_line, final_right_line 


def canny(image):
  gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
  blur = cv2.GaussianBlur(gray, (5, 5),0)
  canny = cv2.Canny(blur, 50, 150)
  return canny

def display_lines(image, lines_to_draw):
  if lines_to_draw is not None: 
    for line in lines_to_draw:
      if line is not None: 
        x1, y1, x2, y2 = line.reshape(4)
        cv2.line(image, (x1, y1), (x2, y2), (255, 0, 0), 2)

: 

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from sort import *
import math


cap = cv2.VideoCapture(r"C:\Users\SUNNY DAVID\Downloads\nD_10.mp4")

model=YOLO('../Yolo-Weights/yolov8n.pt')

classNames = model.names

def region_of_interest(image):
    h, w, _ = image.shape
    roi_x1 = int(0.35 * w)
    roi_y1 = int(0.4 * h)
    roi_x2 = int(0.6 * w)
    roi_y2 = int(h)
    polygons= np.array([[(roi_x1, roi_y1), (roi_x2, roi_y1), (roi_x2, roi_y2),(roi_x1,roi_y2)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, (255,255,255))
    masked_image = cv2.bitwise_and(image, mask)
    limits = [roi_x1,roi_y1 ,roi_x2 ,roi_y2]
    limits= np.array([roi_x1,roi_y1,roi_x2,roi_y2],np.int32)
    return masked_image,limits

tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)
limits=np.array(4)
totalCount = []    
    
while True:
    success, frame = cap.read()
    if not success:
        break

    img ,limits= region_of_interest(frame)
    
    results = model(img, stream=True)
    detections = np.empty((0, 5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            cx = int((x1 + x2) / 2)
            cy = int((y1 + y2) / 2)

            if (currentClass in ["car", "truck", "bus", "motorbike"] and conf > 0.3):
                w_box, h_box = x2 - x1, y2 - y1
                cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)        
                cv2.putText(img, f'{currentClass} {conf:.2f}', (max(0, x1), max(35, y1)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,355), 2, cv2.LINE_AA)
                cv2.circle(img, (cx, cy), 4, (0, 0, 255), -1)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
 
    resultsTracker = tracker.update(detections)

    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        if limits[0] < cx < limits[2] and limits[1] < cy < limits[3]:
            if totalCount.count(id) == 0:
                totalCount.append(id)
            if cy > limits[3]-1 and cx > limits[0]+100 and cx < limits[2]-100:
                cv2.putText(img,"Collision Warning!",(255,180),cv2.FONT_HERSHEY_PLAIN,5,(50,50,255),8)

    cv2.putText(img,str(len(totalCount)),(255,100),cv2.FONT_HERSHEY_PLAIN,5,(50,50,255),8)
    

    cimg=canny(img)
    lines = cv2.HoughLinesP(cimg,1,np.pi/180, 100, np.array([]), maxLineGap=50)
    left_lane_coords, right_lane_coords = average_slope_intercept(img, lines)
  
    lines_for_display = []
    if left_lane_coords is not None:
        lines_for_display.append(left_lane_coords)
    if right_lane_coords is not None:
        lines_for_display.append(right_lane_coords)

    
    if lines_for_display:
      display_lines(img, np.array(lines_for_display))   
    cv2.polylines(img, [np.array([(left_lane_coords[2],left_lane_coords[3]),(right_lane_coords[2],right_lane_coords[3]),(right_lane_coords[0],right_lane_coords[1]),(left_lane_coords[0],left_lane_coords[1])])], True, (0,255,0), 3)
    lx=(left_lane_coords[2]+right_lane_coords[2])//2
    ly=int(img.shape[0]*0.7)
    lane_center = (lx,ly)
    cv2.circle(img, lane_center, 4, (255, 0, 0), -1)
    cv2.imshow("Vehicle Detection with ROI", img)    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()